In [ ]:
import re

import pandas as pd
from pathlib import Path
from ploting import *

In [ ]:
root_path = "guyot"

In [ ]:
time_re = re.compile("apply - inverse mass -.*\|\s+(\d+)\s+\|\s+(\d+(\.\d+)?)")

In [ ]:
iter_re = re.compile("DEAL:.*::iterations:\s*(\d+)")

In [ ]:
batch_iter_re = re.compile("\[(\d+)\|(\d+)\|")

In [ ]:
def config_data(filename: str):
    items = filename.replace(".out", "").split("-")
    return dict((key, value) for key, value in [item.split("_", maxsplit=1) for item in items])

In [ ]:
data = []
for file in Path(f"{root_path}/data").iterdir():
    with open(file, "r") as input:
        if t_m := time_re.search(input.read()):
            data.append({**config_data(file.name),
                         "time": t_m.group(2), "timesteps": t_m.group(1)})
df = pd.DataFrame(data).astype({"dim": int, "ncells": int, "degree": int, "timesteps": int, "time": float}).drop(columns=["cpu", "dryrun", "fmt", "refinement", "dim", "batchsize", "exec", "gpu"])
df

In [ ]:
file = "data/ncells_100-batchsize_full-cpu_False-dim_2-dryrun_False-exec_cuda-fmt_csr-gpu_True-degree_5-refinements_0-solver_batched_iterative-testcase_1.out"
with open(file, "r") as input:
    iters = []
    for l in input:
        if i_m:= batch_iter_re.search(l):
            iters.append(pd.Series(map(int, l[i_m.end():-2].split(","))).describe().T)
    print(pd.DataFrame(iters))

In [ ]:
t = pd.DataFrame(iters)
t.quantile(0.9)

In [ ]:
df['time_normalized'] = df.time / df.timesteps / df.ncells
df

In [ ]:
df0 = df[df.testcase == "0"]
df1 = df[df.testcase == "1"]

In [ ]:
t1 = df1.set_index(["ncells", "degree", "solver"]).time_normalized.sort_index()
t0 = df0.set_index(["ncells", "degree", "solver"]).time_normalized.sort_index()

In [ ]:
t1.unstack([1, 2])[5]

In [ ]:
fig, axes = plt.subplots(1, 3)
data = t1.unstack([1, 2])
data[3].plot(ax=axes[0], logx=True, logy=True)
axes[0].set_title("P3")
data[5].plot(ax=axes[1], logx=True, logy=True)
axes[1].set_title("P5")
data[7].plot(ax=axes[2], logx=True, logy=True)
axes[2].set_title("P7")

In [ ]:
fig, axes = plt.subplots(1, 3)
data = t0.unstack([1, 2])
data[3].plot(ax=axes[0], logx=True, logy=True)
axes[0].set_title("P3")
data[5].plot(ax=axes[1], logx=True, logy=True)
axes[1].set_title("P5")
data[7].plot(ax=axes[2], logx=True, logy=True)
axes[2].set_title("P7")

In [ ]:
a = pd.Series([1,2,3])
b = pd.Series([4,5,6, 7])
p = pd.DataFrame({"a": a, "b": b})
p

In [ ]:
p.columns